In [111]:
import numpy as np
import sympy as sy

A function to calculate the maximal axial force in the internal reinforcement is defined according to the equation (4) of the paper.

In [112]:
def calc_Rn(F_V, F_M, Ft90, beta, d, w):
    """Calculate the axial force in the reinforcement

    :F_V: portion of the force associated with the shear force
    :F_M: portion of the force associated witht the moment 
    :Ft90: tensile vertical force acc. to the German Nationa Annex to EN 1995-1-1
    :beta: inclination angle of the rod (in degrees). beta=0° --> vertical position
    :d: diameter of the internal reinforcement
    :w: width of the glulam beam
    
    :returns: maximal axial force of the reinforcement

    """
    # Set coefficients
    c1 = 1.2
    c2 = 0.012
    c3 = 0.36
    # Calculate sinus and cosinus
    cos_B = np.cos( np.radians(beta) )
    sin_B = np.sin( np.radians(beta) )
    # apply the equation
    R_n = ( (F_V * c1 + F_M * c2 ) * sin_B * np.sqrt(d/w) + Ft90 * c3 * d/np.sqrt(w) ) / cos_B

    return R_n

The parameters of the problem are defined in the next cell

In [113]:
# Dimensions
# Beam:
# - width
w = 120.
# - heigth
h = 450.

# Hole:
# - diameter of the hole
hd = 135.
# - distance from the left-most support to the left edge of the hole
lA = 720.
# - area of the cross-section
A = w*h

h_r = (h - hd)/2. + 0.15 * hd

# Mechanical properties
I = h**3 * w / 12.

# Define symbolic variables
x = sy.Symbol('x')
y = sy.Symbol('y')

# Load
F = 126*2. * 1e3 # N
# Shear force
V = F/2.
# Moment at position 'x' (starting from the left-most support)
M = V * x

In [114]:
# Maximal load (withouth hole)
# - bending strength of the glulam (assumed GL32h)
f_m = 32.
# - maximal moment on the full cross-section
M_max = f_m * I / (h*0.5)
print('M_max = {m:1.0f} kN m'.format(m=M_max*1e-6))

# - shear strength
f_v = 3.5
# - maximal shear force on the full cross-section
V_max = f_v * A / 1.5
print('V_max = {v:1.0f} kN'.format(v=V_max*1e-3))


M_max = 130 kN m
V_max = 126 kN


In [115]:
# Section forces at both sides of the hole
V_1 = V
V_2 = V

# - moment at the left side of the hole
M_1 = M.subs(x, lA)
# - moment at the right side of the hole
M_2 = M.subs(x, lA+hd)

print("V_1 = {v1:1.1f} kN".format(v1=V_1*1e-3))
print("M_1 = {m1:1.1f} kN mm".format(m1=M_1*1e-6))

print("V_2 = {v2:1.1f} kN".format(v2=V_2*1e-3))
print("M_2 = {m2:1.1f} kN mm".format(m2=M_2*1e-6))

V_1 = 126.0 kN
M_1 = 90.7 kN mm
V_2 = 126.0 kN
M_2 = 107.7 kN mm


Calculate the vertical tensile force $F_{t,90}$ according to equations (1), (2) and (3)

In [116]:
F_tV_1 = (V_1 * 0.7*hd)/(4. * h) * (3 - (0.7*hd**2)/(h**2))
F_tM_1 = 0.008 * M_1 /h_r 
F_t90_1 = F_tV_1 + F_tM_1

F_tV_2 = (V_2 * 0.7*hd)/(4. * h) * (3 - (0.7*hd**2)/(h**2))
F_tM_2 = 0.008 * M_2 /h_r 
F_t90_2 = F_tV_2 + F_tM_2

print("F_t90_1 = {f1:1.2f} kN".format(f1=F_t90_1*1e-3))
print("F_t90_2 = {f2:1.2f} kN".format(f2=F_t90_2*1e-3))

F_t90_1 = 23.51 kN
F_t90_2 = 24.28 kN


Calculate the components $F_{r,V}$ according to Eq. (5) and $F_{r,M}$ according to Eq. (6)

In [122]:
# reinf used
d = 12. # mm
beta = 45. # degree
ld = 180. # mm

y_0 = hd*0.5*0.7 - ld * np.cos(np.radians(beta))
# Calculate F_rV
shear_1 = V_1 / (2.*I) * (h**2/4. - y**2)
F_rV_1 = sy.integrate(shear_1 * w, (y, -hd*0.5*0.7, -y_0 ))

shear_2 = V_2 / (2.*I) * (h**2/4. - y**2)
F_rV_2 = sy.integrate(shear_2 * w, (y, y_0, hd*0.5*0.7 ))

# Calculate F_rM
bending_1 = -M_1 * y / I
F_rM_1 = sy.integrate(bending_1 * w, (y, -h/2., -hd*0.5*0.7))

bending_2 = -M_2 * y / I
F_rM_2 = sy.integrate(bending_2 * w, (y, hd*0.5*0.7, h/2.))

print("F_rV_1 = {v1:1.1f} kN".format(v1=F_rV_1*1e-3))
print("F_rM_1 = {m1:1.1f} kN".format(m1=F_rM_1*1e-3))

print("F_rV_2 = {v2:1.1f} kN".format(v2=F_rV_2*1e-3))
print("F_rM_2 = {m2:1.1f} kN".format(m2=F_rM_2*1e-3))

# Calculate the maximal axial force in each reinforcement using the function
# defined at the beginning of this notebook
F_r_1 = calc_Rn(F_V=F_rV_1, F_M=F_rM_1, Ft90=F_t90_1, d=d, w=w, beta=beta)
F_r_2 = calc_Rn(F_V=F_rV_2, F_M=F_rM_2, Ft90=F_t90_2, d=d, w=w, beta=beta)

print('--------------------')
print("F_r_1 = {f1:1.1f} kN".format(f1=F_r_1*1e-3))
print("F_r_2 = {f2:1.1f} kN".format(f2=F_r_2*1e-3))


F_rV_1 = 56.8 kN
F_rM_1 = 289.1 kN
F_rV_2 = 56.8 kN
F_rM_2 = -343.3 kN
--------------------
F_r_1 = 35.8 kN
F_r_2 = 33.8 kN


Verify the shear stresses at the bonding surface of the reinforcement

In [118]:
# Shear stresses at bonding surface
f_k1 = 4.0
Ar = np.pi * d**2 / 4.
peri_r = np.pi * d * ld

# maxumal shear stress in the bounding surface in each reinforcement
tau_r_1 = F_r_1 / peri_r
tau_r_2 = F_r_2 / peri_r

# usage factors
fact_1 = tau_r_1 / f_k1
fact_2 = tau_r_2 / f_k1

print('fact_1 = {t:1.2f}/{f:1.2f} = {fac:1.2f}'.format(t=tau_r_1, f=f_k1, fac=fact_1))
print('fact_2 = {t:1.2f}/{f:1.2f} = {fac:1.2f}'.format(t=tau_r_2, f=f_k1, fac=fact_2))

fact_1 = 4.99/4.00 = 1.25
fact_2 = 4.70/4.00 = 1.17


Verify the yielding stress of the reinforcements under the calculated maximal axial force

In [119]:
# yielding stress
f_y = 500.

# maximal axial stress in each reinforcement
s_1 = F_r_1 / Ar
s_2 = F_r_2 / Ar

# usage factors
fact_1 = s_1 / f_y
fact_2 = s_2 / f_y

print('fact_1 = {t:1.2f}/{f:1.2f} = {fac:1.2f}'.format(t=s_1, f=f_y, fac=fact_1))
print('fact_2 = {t:1.2f}/{f:1.2f} = {fac:1.2f}'.format(t=s_2, f=f_y, fac=fact_2))

fact_1 = 299.27/500.00 = 0.60
fact_2 = 281.83/500.00 = 0.56
